In [1]:
import pandas as pd

In [2]:
df_raw = pd.read_excel(r'M:\Data\OnBoard\Data and Reports\Napa Vine\2019\Napa Vine-FINAL Data-Client_01212021.xlsx',
                       sheet_name = 'Main Data')

In [3]:
print(list(df_raw))
df = df_raw.copy()

df.dropna(how='all', inplace=True)

print(df.shape)

# check duplicates
print('The data has {} records, {} unique CCGID'.format(df.shape[0], len(df.CCGID.unique())))
print('If the data has duplicates records: {}'.format(not(df[df.duplicated()].shape[0] == 0)))

df.rename(columns= {'CCGID': 'ID'}, inplace=True)

# use interview_start_time as time_string
df['survey_time'] = df['interview_start_time']

['sys_RespNum', 'CCGID', 'interview_start_time', 'interview_end_time', 'day_of_week', 'date', 'DTYPE', 'RUNID', 'Route', 'Dir', 'Strata', 'wcode', 'weight', 'tweight', 'Lang', 'From', 'To', 'FromSchool', 'ToSchool', 'FromUni', 'ToUni', 'Start_lat', 'Start_lon', 'End_lat', 'End_lon', 'ToBusMode1', 'ToBusMode2', 'ToBusMode3', 'ToBusMode4', 'GetFirstBus', 'totalbusBEFORE', 'Route_Before_1', 'Route_Before_2', 'Route_Before_3', 'Route_Before_4', 'System_Before_1', 'System_Before_2', 'System_Before_3', 'System_Before_4', 'FromBusMode1', 'FromBusMode2', 'FromBusMode3', 'FromBusMode4', 'FromLastBus', 'totalbusAFTER', 'Route_After_1', 'Route_After_2', 'Route_After_3', 'Route_After_4', 'System_After_1', 'System_After_2', 'System_After_3', 'System_After_4', 'fare', 'farecat', 'Alternatives', 'UseVineMoreFrequency', 'UseVineMoreDestination', 'UseVineMoreEarlier', 'UseVineMoreLater', 'cars', 'hh', 'hhwork', 'birthyear', 'Age', 'hisp', 'ETH1', 'ETH2', 'ETH3', 'ETH4', 'income', 'langhh', 'engspk', 'l

In [4]:
# build 'access_mode' and 'egress_mode' variables based on relevant fields

ToBusMode_dict = {1: 'walk',
                  2: 'public transit',
                  3: 'Drove alone and parked', 
                  4: 'Drove or rode with others and parked',
                  5: 'Dropped off by someone (NOT a service)', 
                  6: 'Dropped off by Uber, Lyft or a similiar service', 
                  7: 'Bike', 
                  8: 'Taxi', 
                  9: 'Other', 
                  10: 'Company Shuttle'}


FromBusMode_dict = {1: 'walk',
                    2: 'public transit',
                    3: 'Drive alone in vehicle parked nearby', 
                    4: 'Drive or ride with others in vehicle parked nearby',
                    5: 'Picked up by someone (NOT a service)', 
                    6: 'Picked up using Uber, Lyft, or similar service', 
                    7: 'Bike', 
                    8: 'Taxi', 
                    9: 'Other', 
                    10: 'Company Shuttle'}

FromLastBus_dict = {1: 'walk', 
                    2: 'Drive alone in vehicle parked nearby', 
                    3: 'Drive or ride with others in vehicle parked nearby', 
                    4: 'Picked up by someone (NOT a service)', 
                    5: 'Picked up using Uber, Lyft, or similar service', 
                    6: 'Bike', 
                    7: 'Taxi', 
                    8: 'Other',
                    9: 'Company Shuttle'}

df['ToBusMode1'].fillna(0, inplace=True)
df['ToBusMode1'] = df['ToBusMode1'].apply(lambda x: int(x))
df['ToBusMode1'] = df['ToBusMode1'].map(ToBusMode_dict)

df['FromBusMode1'].fillna(0, inplace=True)
df['FromBusMode1'] = df['FromBusMode1'].apply(lambda x: int(x))
df['FromBusMode1'] = df['FromBusMode1'].map(FromBusMode_dict)

df['FromLastBus'].fillna(0, inplace=True)
df['FromLastBus'] = df['FromLastBus'].apply(lambda x: int(x))
df['FromLastBus'] = df['FromLastBus'].map(FromLastBus_dict)


df['access_mode'] = df['ToBusMode1']
print('access_mode stats: \n{}'.format(df.access_mode.value_counts()))

df['egress_mode'] = df['FromBusMode1']
df.loc[df.FromBusMode1 == 'public transit', 'egress_mode'] = df['FromLastBus']
print('egress_mode stats: \n{}'.format(df.egress_mode.value_counts()))

access_mode stats: 
walk                                               277
Dropped off by someone (NOT a service)              24
Bike                                                16
Drove or rode with others and parked                12
Dropped off by Uber, Lyft or a similiar service      5
Drove alone and parked                               2
Taxi                                                 2
Name: access_mode, dtype: int64
egress_mode stats: 
walk                                                  278
Picked up by someone (NOT a service)                   17
Bike                                                   16
Drive alone in vehicle parked nearby                   13
Picked up using Uber, Lyft, or similar service          9
Drive or ride with others in vehicle parked nearby      4
Taxi                                                    1
Name: egress_mode, dtype: int64


In [5]:
# code transfer system and create columns for 'system + route' which will be used for canonical_route mapping

system_dict = {1: 'BART',
               2: 'Napa Vine',
               3: 'County Connection',
               4: 'FAST (Fairfield-Suisun Transit)',
               5: 'SolTrans (Solano County Transit)',
               6: 'Sonoma County Transit',
               7: 'Tri Delta Transit',
               8: 'Vacaville City Coach',
               9: 'AC Transit',
               10: 'Caltrain',
               11: 'Golden Gate Transit',
               12: 'Golden Gate Ferry',
               13: 'SamTrans',
               14: 'San Francisco Bay Ferry',
               15: 'San Francisco Muni',
               16: 'VTA',
               17: 'WestCat',
               18: 'WHEELS',
               19: 'Other (type in next to route)',
               20: 'Lake Transit',
               21: 'Capitol Corridor'}

for i in ['System_Before_1', 'System_Before_2', 'System_Before_3', 'System_Before_4',
          'System_After_1', 'System_After_2', 'System_After_3', 'System_After_4']:
    #print(i)
    #print(df[i])
    df[i] = df[i].fillna(0)
    df[i].replace(to_replace = ' ', value = 0, inplace=True)
    df[i] = df[i].apply(lambda x: int(x))
    df[i+'_temp'] = df[i].map(system_dict)
    #print(df[i+'_temp'])

df['first_route_before_survey_board'] = df['System_Before_1_temp'] + '___' + df['Route_Before_1'].astype(str)
df['second_route_before_survey_board'] = df['System_Before_2_temp'] + '___' + df['Route_Before_2'].astype(str)
df['third_route_before_survey_board'] = df['System_Before_3_temp'] + '___' + df['Route_Before_3'].astype(str)
df['first_route_after_survey_alight'] = df['System_After_1_temp'] + '___' + df['Route_After_1'].astype(str)
df['second_route_after_survey_alight'] = df['System_After_2_temp'] + '___' + df['Route_After_2'].astype(str)
df['third_route_after_survey_alight'] = df['System_After_3_temp'] + '___' + df['Route_After_3'].astype(str)

In [6]:
# create new field to represent complete survey route info (route + direction)
df['route_dir'] = df.Route.astype(str) + '_' + df.Dir
df.route_dir.unique()

array(['29.0_NORTH', '2.0_LOOP', '3.0_LOOP', '29.0_SOUTH', '10.0_SOUTH',
       '1.0_LOOP', '10.0_NORTH', '4.0_LOOP', '11.0_NORTH', '8.0_SOUTH',
       '21.0_EAST', '5.0_LOOP', '8.0_NORTH', '21.0_WEST', '11.0_SOUTH',
       '7.0_LOOP', '6.0_LOOP'], dtype=object)

In [7]:
race_dict = {1: 'American Indian/Alaska Native',
             2: 'Native Hawaiian/Pacific Islander',
             3: 'Black/African American',
             4: 'White/Caucasian',
             5: 'Asian',
             6: 'Other',
             7: 'Hispanic',
             8: 'Mixed',
             9: 'Persian/Arab/North African/Middle Eastern',
             0: 'Refused',
             10: 'NA'}

for i in ['ETH1', 'ETH2', 'ETH3', 'ETH4']:
    df[i] = df[i].fillna(10)
    df[i].replace(to_replace = ' ', value = 10, inplace=True)
    df[i] = df[i].apply(lambda x: int(x))
    df[i+'_temp'] = df[i].map(race_dict)
    
df['ETH_concat'] = df['ETH1_temp'] + '_' + df['ETH2_temp'] + '_' + df['ETH3_temp'] + '_' + df['ETH4_temp']
print(list(df['ETH_concat'].unique()))

['American Indian/Alaska Native_NA_NA_NA', 'Hispanic_NA_NA_NA', 'White/Caucasian_NA_NA_NA', 'Black/African American_NA_NA_NA', 'White/Caucasian_Hispanic_NA_NA', 'Asian_NA_NA_NA', 'Mixed_NA_NA_NA', 'American Indian/Alaska Native_Hispanic_NA_NA', 'Hispanic_American Indian/Alaska Native_NA_NA', 'American Indian/Alaska Native_White/Caucasian_Hispanic_NA', 'American Indian/Alaska Native_Black/African American_White/Caucasian_Asian', 'Mixed_Hispanic_NA_NA', 'Black/African American_White/Caucasian_NA_NA', 'Hispanic_White/Caucasian_NA_NA', 'White/Caucasian_Hispanic_American Indian/Alaska Native_NA', 'Native Hawaiian/Pacific Islander_NA_NA_NA', 'Black/African American_White/Caucasian_Hispanic_NA', 'Native Hawaiian/Pacific Islander_Black/African American_Asian_NA', 'American Indian/Alaska Native_White/Caucasian_NA_NA', 'Refused_NA_NA_NA', 'Asian_Hispanic_NA_NA', 'Black/African American_Hispanic_NA_NA', 'Persian/Arab/North African/Middle Eastern_NA_NA_NA', 'Native Hawaiian/Pacific Islander_White/

In [8]:
df['hisp_from_ETH'] = 'non-hisp'
df.loc[df.ETH_concat.str.contains('Hispanic',na=False), 'hisp_from_ETH'] = 'hisp'

# check consistency between 'hisp' derived from ETH and the original 'hisp' variable and make corrections

# records where ETH indicates 'hispanic' but 'hisp' is 1 (not hispanic)
display(df.loc[(df.hisp_from_ETH == 'hisp') & (df.hisp != 2)][['ID','hisp','hisp_from_ETH',
                                                                 'ETH1', 'ETH2', 'ETH3', 'ETH4']])
# make hisp=2 for records where hisp_from_ETH == 'hisp'
df.loc[df.hisp_from_ETH == 'hisp', 'hisp'] = 2

# records where ETH doesn't indicate 'hispanic' but 'hisp' is 2 (is hispanic)
# for these, keep the '2' value because the surveyed may leave 'ETH-hispanic' out when they feel
# they have already provided the information in 'hisp'
display(df.loc[(df.hisp_from_ETH != 'hisp') & (df.hisp == 2)][['ID','hisp','hisp_from_ETH',
                                                                 'ETH1', 'ETH2', 'ETH3', 'ETH4']])



,ID,hisp,hisp_from_ETH,ETH1,ETH2,ETH3,ETH4
78,101.0,1,hisp,4,7,1,10


,ID,hisp,hisp_from_ETH,ETH1,ETH2,ETH3,ETH4
0,9.0,2,non-hisp,1,10,10,10
4,13.0,2,non-hisp,4,10,10,10
193,9010.0,2,non-hisp,4,10,10,10
194,9011.0,2,non-hisp,4,10,10,10
195,9012.0,2,non-hisp,0,10,10,10
197,9020.0,2,non-hisp,4,10,10,10
203,9028.0,2,non-hisp,0,10,10,10
204,9029.0,2,non-hisp,0,10,10,10


In [9]:
# create race_dmy_xx

df['race_dmy_ind'] = 0
df.loc[df.ETH_concat.str.contains('American Indian/Alaska Native',na=False), 'race_dmy_ind'] = 1

df['race_dmy_hwi'] = 0
df.loc[df.ETH_concat.str.contains('Native Hawaiian/Pacific Islander',na=False), 'race_dmy_hwi'] = 1

df['race_dmy_blk'] = 0
df.loc[df.ETH_concat.str.contains('Black/African American',na=False), 'race_dmy_blk'] = 1

df['race_dmy_wht'] = 0
df.loc[df.ETH_concat.str.contains('White/Caucasian',na=False), 'race_dmy_wht'] = 1

df['race_dmy_asn'] = 0
df.loc[df.ETH_concat.str.contains('Asian',na=False), 'race_dmy_asn'] = 1

df['race_dmy_mdl_estn'] = 0
df.loc[df.ETH_concat.str.contains('Persian/Arab/North African/Middle Eastern',na=False), 'race_dmy_mdl_estn'] = 1

In [10]:
# drop interim columns and export
df.drop(columns = ['System_Before_1_temp', 'System_Before_2_temp', 'System_Before_3_temp',
                   'System_Before_4_temp', 'System_After_1_temp', 'System_After_2_temp',
                   'System_After_3_temp', 'System_After_4_temp', 'ETH1_temp', 'ETH2_temp',
                   'ETH3_temp', 'ETH4_temp', 'ETH_concat', 'hisp_from_ETH'], inplace=True)

df.to_csv(r'M:\Data\OnBoard\Data and Reports\Napa Vine\2019\As CSV\Napa Vine_FINAL Data_addCols_NO POUND OR SINGLE QUOTE.csv', index=False)

In [11]:
# bring in standard dictionary to check field consistency

# dictionary for Napa Vine survey
var = pd.read_csv(r'M:\Data\OnBoard\Data and Reports\Napa Vine\2019\variable_dictionary.csv',
                  encoding = "ISO-8859-1", engine='python')

# standard dictionary
var_standard = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\onboard-surveys\make-uniform\production\Dictionary for Standard Database.csv')
var_standard.columns = [x+'_s' for x in list(var_standard)]

# merge
var_merge = var.merge(var_standard, left_on='Generic_Variable', right_on='Generic_Variable_s', how='outer')

# check if 'Generic_Variable' in Napa Vine dictionary matches the standard 'Generic_Variable'. chk1 should be empty
chk1 = var_merge.loc[(var_merge.Generic_Variable.notnull()) & (var_merge.Generic_Variable_s.isnull())]
print('Generic_Variable that shouldn not exit:')
print(chk1.Generic_Variable.unique())
print()

# check if columns names in survey data matches 'Survey_Variable' in Napa Vine dictionary.
# the following loops should not include variables that are needed for standardization

for i in var.loc[var.Generic_Variable.notnull()]['Survey_Variable']:
    if i not in list(df):
        print(i)
        
for i in list(df):
    if i not in list(var.Survey_Variable):
        print(i)

Generic_Variable that shouldn not exit:
['interview_start_time' 'interview_end_time']

sys_RespNum
RUNID
Route
com
access_mode
egress_mode


In [12]:
var_clean = var[['operator', 'Survey_year', 'Survey_Variable', 'Survey_Response', 
                 'Generic_Variable', 'Generic_Response']].drop_duplicates()
var_clean = var_clean.loc[var_clean.Generic_Variable.notnull()]

# export
# var_clean.to_csv(r'M:\Data\OnBoard\Data and Reports\Napa Vine\2019\vars_for_standard_dictionary.csv', index=False)

In [13]:
# create raw transfer routes file
routes = pd.DataFrame(columns = ['survey_name'])
for i in ['first_route_before_survey_board', 'second_route_before_survey_board',
          'third_route_before_survey_board', 'first_route_after_survey_alight', 
          'second_route_after_survey_alight', 'third_route_after_survey_alight']:
    route_unique = df[[i]]
    route_unique.columns = ['survey_name']
    routes = pd.concat([routes, route_unique])

routes_clean = routes.loc[routes.survey_name.notnull()]
routes_clean.drop_duplicates(inplace=True)
routes_clean['survey'] = 'Napa Vine'
routes_clean['survey_year'] = 2019

print(routes_clean.shape)
# routes_clean[['survey','survey_year','survey_name']].to_csv(r'C:\Users\ywang\Documents\R\OnboardSurvey_2020Oct_yq\Data and Reports\Napa Vine\2019\all_routes_raw.csv', index=False)

(55, 3)


C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https